In [1]:
from flask import Flask,jsonify,render_template
from flask_cors import CORS
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [9]:
app = Flask(__name__)
CORS(app)
with open("../restaurant_data.json", 'r') as file:
    data = json.load(file)

restaurnt_df = pd.DataFrame(data)

def add_vote_to_db(vote):
    # session = Session(bind=engine)
    # session.update({vote: vote+1}) #not how it wokrs but i cant be bothered rn
    print(f"added {vote} to db")
    
#--SQL STUFF
engine = create_engine("sqlite:///Resources/votes.sqlite")
# base = automap_base()
# base.prepare(autoload_with=engine)

#-- 

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/map")
def dontwait():
    return render_template('map.html')

@app.route("/chart")
def justkiddingyoucanwait():
    return render_template('chart.html')

@app.route("/btc")
def nvm():
    return render_template('btc.html')

@app.route("/vote<vote>")
def noway(vote):
    # print(vote)
    add_vote_to_db(vote)
    return vote
    
@app.route("/voteTot")
def jose():
    return json.dumps("Woah you are crazy")



if(__name__=="__main__"):
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Apr/2023 19:32:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 19:32:07] "GET / HTTP/1.1" 200 -
